In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [23]:
data=pd.read_csv('casestudy.csv',index_col=0)

In [24]:
data.head()

,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015


In [41]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext() 

In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [43]:
from pyspark.sql.types import *

In [53]:
data = spark.read.csv('casestudy.csv',inferSchema=True,header=True)

In [54]:
data.createOrReplaceTempView("data")

There is 1 dataset(csv) with 3 years worth of customer orders. There are 4 columns in the csv dataset: index, CUSTOMER_EMAIL(unique identifier as hash), Net_Revenue, and Year.

For each year we need the following information:

* Total revenue for the current year
* New Customer Revenue e.g. new customers not present in previous year only
* Existing Customer Growth, Revenue of existing customers for current year – Revenue of existing customers from existing year
* Revenue lost from attrition
* Existing Customer Revenue Current Year
* Existing Customer Revenue Prior Year
* Total Customers Current Year
* Total Customers Previous Year
* New Customers
* Lost Customers

## Total revenue for the current year

In [13]:
total_revenue_current_year=data[['net_revenue','year']].groupby('year').sum().reset_index().rename(columns={'net_revenue':'total_revenue'})
total_revenue_current_year

,year,total_revenue
0,2015,29036749.19
1,2016,25730943.59
2,2017,31417495.03


In [55]:
spark.sql('''
SELECT year, sum(net_revenue) as total_revenue
  FROM data
  GROUP BY year
''').show()

+----+--------------------+
|year|       total_revenue|
+----+--------------------+
|2015|2.9036749189999793E7|
|2016|2.5730943590000015E7|
|2017|3.1417495030000173E7|
+----+--------------------+



## New Customer Revenue

For year 2015, we can't calculate new customer revenue, for we don't know the data in 2014.

For year 2016, new customers are customers who didn't purchase in 2015 but purchased in 2016.

For year 2017, new customers are customers who didn't purchase in 2016 but purchased in 2017.

In [70]:
spark.sql('''
SELECT d.year, SUM(d.net_revenue)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NULL AND d1.year!=2015) d
GROUP BY d.year
''').show()

+----+--------------------+
|year|    sum(net_revenue)|
+----+--------------------+
|2016|1.8245491010000005E7|
|2017|       2.877623504E7|
+----+--------------------+



## Existing Customer Growth, Revenue of existing customers for current year – Revenue of existing customers from existing year

For year 2015, we can't calculate Existing Customer Growth.

For year 2016, Existing Customer are customers who purchased in 2015 and also purchased in 2016.

For year 2017, Existing Customer are customers who purchased in 2016 and also purchased in 2017.

In [105]:
spark.sql('''
SELECT d.year, SUM(d.net_revenue)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NOT NULL AND d1.year!=2015) d
GROUP BY d.year
''').show()

+----+------------------+
|year|  sum(net_revenue)|
+----+------------------+
|2016| 7485452.579999998|
|2017|2641259.9900000016|
+----+------------------+



## Revenue lost from attrition

To calculate revenue lost from attrition:

1. consider client who purchase in last year, but not purchase in this year as lost clients.

2. in this year, assume lost client purchase the same amount as last year

For year 2015, we can't calculate revenue lost from attrition, for we don't know the data in 2014.

For year 2016, lost customers are customers who purchased in 2015 but didn't purchase in 2016. We calculate the revenue for lost customers in 2015.

For year 2017, lost customers are customers who purchased in 2016 but didn't purchase in 2017. We calculate the revenue for lost customers in 2016.

In [107]:
spark.sql('''
SELECT d.year, SUM(d.net_revenue)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year-1
  WHERE d2.customer_email IS NULL AND d1.year!=2017) d
GROUP BY d.year
''').show()

+----+--------------------+
|year|    sum(net_revenue)|
+----+--------------------+
|2015|2.1571632070000015E7|
|2016|2.3110294939999998E7|
+----+--------------------+



Thus, for year 2016, Revenue lost from attrition is 21571632.070000015.

For year 2017, Revenue lost from attrition is 23110294.939999998.

## Existing Customer Revenue Current Year

For year 2015, we can't calculate Existing Customer Growth.

For year 2016, Existing Customer are customers who purchased in 2015 and also purchased in 2016.

For year 2017, Existing Customer are customers who purchased in 2016 and also purchased in 2017.

In [112]:
spark.sql('''
SELECT d.year, SUM(d.net_revenue)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NOT NULL AND d1.year!=2015) d
GROUP BY d.year
''').show()

+----+------------------+
|year|  sum(net_revenue)|
+----+------------------+
|2016| 7485452.579999998|
|2017|2641259.9900000016|
+----+------------------+



## Existing Customer Revenue Prior Year

For year 2015, we can't calculate Existing Customer Revenue Prior Year.

For year 2016, Existing Customer are customers who purchased in 2015 and also purchased in 2016.

For year 2017, Existing Customer are customers who purchased in 2016 and also purchased in 2017.

In [114]:
spark.sql('''
SELECT d.year, SUM(d.net_revenue)
FROM
(SELECT d2.customer_email, d2.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NOT NULL AND d1.year!=2015) d
GROUP BY d.year
''').show()

+----+------------------+
|year|  sum(net_revenue)|
+----+------------------+
|2016|7465117.1199999945|
|2017| 2620648.650000002|
+----+------------------+



## Total Customers Current Year

In [12]:
total_customers_current_year=data[['customer_email','year']].groupby('year').nunique().reset_index().rename(columns={'customer_email':'total_customer'})
total_customers_current_year

,year,total_customer
0,2015,231294
1,2016,204646
2,2017,249987


In [63]:
spark.sql('''
SELECT year, COUNT(DISTINCT customer_email)
  FROM data
  GROUP BY year
''').show()

+----+------------------------------+
|year|count(DISTINCT customer_email)|
+----+------------------------------+
|2015|                        231294|
|2016|                        204646|
|2017|                        249987|
+----+------------------------------+



## Total Customers Previous Year

It's silimiar as Total Customers Current Year.

For year 2015, we can't calculate Total Customers Previous Year, for we don't know the data in 2014.

For year 2016, the Total Customers Previous Year is 231294.

For year 2017, the Total Customers Previous Year is 204646.

For year 2018, the Total Customers Previous Year is 249987.

## New Customers

For year 2015, we can't calculate new customer, for we don't know the data in 2014.

For year 2016, new customers are customers who didn't purchase in 2015 but purchased in 2016.

For year 2017, new customers are customers who didn't purchase in 2016 but purchased in 2017.

In [106]:
spark.sql('''
SELECT d.year, COUNT(DISTINCT customer_email)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NULL AND d1.year!=2015) d
GROUP BY d.year
''').show()

+----+------------------------------+
|year|count(DISTINCT customer_email)|
+----+------------------------------+
|2016|                        145062|
|2017|                        229028|
+----+------------------------------+



In [108]:
spark.sql('''
SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year+1
  WHERE d2.customer_email IS NULL AND d1.year!=2015
''').show()

+--------------------+-----------+----+
|      customer_email|net_revenue|year|
+--------------------+-----------+----+
| abkfhjrhmx@gmail...|      162.2|2017|
| acruircmdh@gmail...|     193.56|2016|
| aedfrodllx@gmail...|     169.17|2017|
| ajuwddxogq@gmail...|      26.66|2017|
| ajwdtypwub@gmail...|     118.07|2017|
| akiqsdaifp@gmail...|     136.51|2017|
| akmswmtkgk@gmail...|       8.37|2016|
| amvidxhcys@gmail...|     159.91|2016|
| anmozpiryh@gmail...|     231.52|2017|
| aogoqhbmbc@gmail...|       6.29|2017|
| aohgiakgpb@gmail...|     178.62|2016|
| aojgacbnzp@gmail...|     227.46|2016|
| apcizyqxlr@gmail...|     197.62|2016|
| asmaezmqgf@gmail...|     162.76|2017|
| atasquacag@gmail...|      16.88|2017|
| atbzzkhcxh@gmail...|      20.15|2017|
| auvjqaoxsh@gmail...|     125.48|2017|
| axdjmivvdy@gmail...|     126.92|2016|
| aymhbujibl@gmail...|     116.06|2016|
| azfbhgvcow@gmail...|     176.88|2016|
+--------------------+-----------+----+
only showing top 20 rows



## Lost Customers

For year 2015, we can't calculate lost customers, for we don't know the data in 2014.

For year 2016, lost customers are customers who purchased in 2015 but didn't purchase in 2016.

For year 2017, lost customers are customers who purchased in 2016 but didn't purchase in 2017.

In [109]:
spark.sql('''
SELECT d.year, COUNT(DISTINCT customer_email)
FROM
(SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year-1
  WHERE d2.customer_email IS NULL AND d1.year!=2017) d
GROUP BY d.year
''').show()

+----+------------------------------+
|year|count(DISTINCT customer_email)|
+----+------------------------------+
|2015|                        171710|
|2016|                        183687|
+----+------------------------------+



Thus, for year 2016, number of lost customers is 171710.

For year 2017, number of lost customers is 183687.

The list of lost customers are listed below. The year is the purchase year.

For example, for customer acruircmdh@gmail.com, the listed year is 2016. Thus, he/she is a lost customer in year 2017.

In [110]:
spark.sql('''
SELECT d1.customer_email, d1.net_revenue,d1.year
  FROM data d1
  LEFT JOIN data d2 on d1.customer_email=d2.customer_email AND d1.year=d2.year-1
  WHERE d2.customer_email IS NULL AND d1.year!=2017
''').show()

+--------------------+-----------+----+
|      customer_email|net_revenue|year|
+--------------------+-----------+----+
| acruircmdh@gmail...|     193.56|2016|
| adxrqqcpnk@gmail...|     179.36|2015|
| agekxcxost@gmail...|      81.99|2016|
| agmwecakcw@gmail...|     168.78|2015|
| agrmxjhfuw@gmail...|      81.92|2016|
| ajcyofdwpx@gmail...|       9.48|2016|
| ajlbrfghkl@gmail...|      16.91|2015|
| akmswmtkgk@gmail...|       8.37|2016|
| alntshbyqu@gmail...|      67.21|2015|
| altvwkiedk@gmail...|      225.7|2015|
| amvidxhcys@gmail...|     159.91|2016|
| anvlqeznxf@gmail...|       4.08|2015|
| aohgiakgpb@gmail...|     178.62|2016|
| aojgacbnzp@gmail...|     227.46|2016|
| aolzejffob@gmail...|     234.19|2015|
| apcizyqxlr@gmail...|     197.62|2016|
| aquhxcswqb@gmail...|      51.42|2015|
| aqzehwqcul@gmail...|      94.26|2015|
| aswrlnaoas@gmail...|      90.59|2015|
| auteybbbtk@gmail...|      57.48|2015|
+--------------------+-----------+----+
only showing top 20 rows

